In [1]:
!python -V

Python 3.9.15


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error


In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi")

<Experiment: artifact_location='./mlruns/4', creation_time=1673782805020, experiment_id='4', last_update_time=1673782805020, lifecycle_stage='active', name='nyc-taxi', tags={}>

In [4]:


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df



In [5]:
df_train = read_dataframe('/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-02.parquet')

In [6]:
len(df_train), len(df_val)

(73908, 61921)

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']


In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.7587151974993125

### Using XgBoost

In [11]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope


In [12]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)


In [13]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=30,   #1000
            evals=[(valid, 'validation')],
            early_stopping_rounds=10   #50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}



In [14]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[18:59:39] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.                                     
[0]	validation-rmse:7.32451                                                                                                                                
[1]	validation-rmse:6.69115                                                                                                                                
[2]	validation-rmse:6.63667                                                                                                                                
[3]	validation-rmse:6.61924                                                                                                                                
[4]	validation-rmse:6.60872                                                                                                                                
[5]	validation-rmse:6.59798                                     

[20]	validation-rmse:8.55810                                                                                                                               
[21]	validation-rmse:8.37783                                                                                                                               
[22]	validation-rmse:8.21497                                                                                                                               
[23]	validation-rmse:8.06837                                                                                                                               
[24]	validation-rmse:7.93653                                                                                                                               
[25]	validation-rmse:7.81800                                                                                                                               
[26]	validation-rmse:7.71039                                    

[10]	validation-rmse:6.62253                                                                                                                               
[11]	validation-rmse:6.61962                                                                                                                               
[12]	validation-rmse:6.61700                                                                                                                               
[13]	validation-rmse:6.61404                                                                                                                               
[14]	validation-rmse:6.61163                                                                                                                               
[15]	validation-rmse:6.60910                                                                                                                               
[16]	validation-rmse:6.60571                                    

[0]	validation-rmse:12.05802                                                                                                                               
[1]	validation-rmse:8.48100                                                                                                                                
[2]	validation-rmse:7.22680                                                                                                                                
[3]	validation-rmse:6.80252                                                                                                                                
[4]	validation-rmse:6.64442                                                                                                                                
[5]	validation-rmse:6.56826                                                                                                                                
[6]	validation-rmse:6.53705                                     

[21]	validation-rmse:6.48935                                                                                                                               
[22]	validation-rmse:6.48977                                                                                                                               
[23]	validation-rmse:6.48656                                                                                                                               
[24]	validation-rmse:6.48452                                                                                                                               
[25]	validation-rmse:6.48413                                                                                                                               
[26]	validation-rmse:6.48397                                                                                                                               
[27]	validation-rmse:6.48325                                    

...

[10]	validation-rmse:6.51295                                                                                                                               
[11]	validation-rmse:6.49963                                                                                                                               
[12]	validation-rmse:6.49253                                                                                                                               
[13]	validation-rmse:6.48527                                                                                                                               
[14]	validation-rmse:6.47953                                                                                                                               
[15]	validation-rmse:6.47595                                                                                                                               
[16]	validation-rmse:6.47128                                    

[0]	validation-rmse:17.67082                                                                                                                               
[1]	validation-rmse:14.92395                                                                                                                               
[2]	validation-rmse:12.80307                                                                                                                               
[3]	validation-rmse:11.18479                                                                                                                               
[4]	validation-rmse:9.96286                                                                                                                                
[5]	validation-rmse:9.05568                                                                                                                                
[6]	validation-rmse:8.38481                                     

[21]	validation-rmse:8.35526                                                                                                                               
[22]	validation-rmse:8.18918                                                                                                                               
[23]	validation-rmse:8.03621                                                                                                                               
[24]	validation-rmse:7.89727                                                                                                                               
[25]	validation-rmse:7.77568                                                                                                                               
[26]	validation-rmse:7.66179                                                                                                                               
[27]	validation-rmse:7.56210                                    

[11]	validation-rmse:6.70664                                                                                                                               
[12]	validation-rmse:6.70674                                                                                                                               
[13]	validation-rmse:6.70723                                                                                                                               
[14]	validation-rmse:6.70707                                                                                                                               
[15]	validation-rmse:6.70565                                                                                                                               
[16]	validation-rmse:6.70605                                                                                                                               
[17]	validation-rmse:6.70511                                    

[1]	validation-rmse:16.68828                                                                                                                               
[2]	validation-rmse:14.93717                                                                                                                               
[3]	validation-rmse:13.46101                                                                                                                               
[4]	validation-rmse:12.22457                                                                                                                               
[5]	validation-rmse:11.19095                                                                                                                               
[6]	validation-rmse:10.33585                                                                                                                               
[7]	validation-rmse:9.63099                                     

## experimenting using autolog

In [17]:
mlflow.xgboost.autolog(disable=True)


In [19]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=30,  #1000
        evals=[(valid, 'validation')],
        early_stopping_rounds=10 #50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")


[19:27:10] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95635
[2]	validation-rmse:16.59103
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29029
[5]	validation-rmse:13.32807
[6]	validation-rmse:12.47571
[7]	validation-rmse:11.72138
[8]	validation-rmse:11.05825
[9]	validation-rmse:10.47535
[10]	validation-rmse:9.96449
[11]	validation-rmse:9.51756
[12]	validation-rmse:9.12641
[13]	validation-rmse:8.78569
[14]	validation-rmse:8.49089
[15]	validation-rmse:8.23445
[16]	validation-rmse:8.01207
[17]	validation-rmse:7.81886
[18]	validation-rmse:7.65169
[19]	validation-rmse:7.50672
[20]	validation-rmse:7.38153
[21]	validation-rmse:7.27299
[22]	validation-rmse:7.17863
[23]	validation-rmse:7.09718
[24]	validation-rmse:7.02603
[25]	validation-rmse:6.96344
[26]	validation-rmse:6.90935
[27]	validation-rmse:6.86285
[28]	validation-rmse:6.82157
[29]	validation-rmse:6.78486


FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'

In [20]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "/home/lambu/Desktop/Project2022_2023/MLOps_Zoomcamp_1/input_raw_data/green_tripdata_2021-01.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

FileNotFoundError: [Errno 2] No such file or directory: 'models/preprocessor.b'